## Load Packages

In [1]:
include("printmat.jl")

printmatDate

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Payoffs and Profits of Options

Call and put profits (at expiration)

$\text{call profit}_{t+m}\ = \max\left(  0,S_{t+m}-K\right) - C$

$\text{put profit}_{t+m}=\max\left(  0,K-S_{t+m}\right)  - P $

The profit of a straddle is the sum of those of a call and a put.

In [3]:
Sm = 0:0.1:10          #possible values of underlying at expiration
K  = 5                 #strike price
C  = 0.4               #call price (just a number that I made up)
P  = 0.4               #put price 

CallProfit = max.(0,Sm-K) .- C
PutProfit  = max.(0,K-Sm) .- P

StraddleProfit = CallProfit + PutProfit

println()

In [4]:
plot(Sm,CallProfit,color=:red,line=(:dash,2),label="call")
plot!(Sm,PutProfit,color=:green,line=(:dot,2),label="put")
plot!(ylim=(-1,5))
title!("Profits of Call and Put Options, strike = $K")
xlabel!("Asset price at expiration")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 Profits of Call and Put Options, strike = 5 
 
 
 Asset price at expiration 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 29.2591,327.208 34.7697,327.208 40.2803,327.208 45.7908,327.208 51.3014,327.208 56.8119,327.208 62.3225,327.208 67.833,327.208 73.3436,327.208 78.8542,327.208 
 84.3647,327.208 89.8753,327.208 95.3858,327.208 100.896,327.208 106.407,327.208 111.918,327.208 117.428,327.208 122.939,327.208 128.449,327.208 133.96,327.208 
 139.47,327.208 144.981,327.208 150.491,327.208 156.002,327.208 161.513,327.208 167.023,327.208 172.534,327.208 178.044,327.208 183.555,327.208 189.065,327.208 
 194.576,327.208 200.086,327.208 205.597,327.208 211.108,327.208 216.618,327.208 222.129,327.208 227.639,327.208 233.15,327.208 238.66,327.208 244.171,327.208 
 249.681,327.208 255.192,327.208 260.703,327.208 266.213,327.208 271.724,327.208 277.234,327.208 282.745,327.208 288.255,327.208 293.766,327.208 299.276,327.208 
 304.787,327.208 310.298,321.732 315.808,316.256 321.319,310.78 326.829,305.304 332.34,299.828 337.85,294.352 343.361,288.875 348.872,283.399 354.382,277.923 
 359.893,272.447 365.403,266.971 370.914,261.495 376.424,256.018 381.935,250.542 387.445,245.066 392.956,239.59 398.467,234.114 403.977,228.638 409.488,223.162 
 414.998,217.685 420.509,212.209 426.019,206.733 431.53,201.257 437.04,195.781 442.551,190.305 448.062,184.828 453.572,179.352 459.083,173.876 464.593,168.4 
 470.104,162.924 475.614,157.448 481.125,151.971 486.635,146.495 492.146,141.019 497.657,135.543 503.167,130.067 508.678,124.591 514.188,119.115 519.699,113.638 
 525.209,108.162 530.72,102.686 536.23,97.2099 541.741,91.7338 547.252,86.2576 552.762,80.7815 558.273,75.3053 563.783,69.8292 569.294,64.353 574.804,58.8768 
 580.315,53.4007 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="1, 2" points="
 29.2591,53.4007 34.7697,58.8768 40.2803,64.353 45.7908,69.8292 51.3014,75.3053 56.8119,80.7815 62.3225,86.2576 67.833,91.7338 73.3436,97.2099 78.8542,102.686 
 84.3647,108.162 89.8753,113.638 95.3858,119.115 100.896,124.591 106.407,130.067 111.918,135.543 117.428,141.019 122.939,146.495 128.449,151.971 133.96,157.448 
 139.47,162.924 144.981,168.4 150.491,173.876 156.002,179.352 161.513,184.828 167.023,190.305 172.534,195.781 178.044,201.257 183.555,206.733 189.065,212.209 
 194.576,217.685 200.086,223.162 205.597,228.638 211.108,234.114 216.618,239.59 222.129,245.066 227.639,250.542 233.15,256.018 238.66,261.495 244.171,266.971 
 249.681,272.447 255.192,277.923 260.703,283.399 266.213,288.875 271.724,294.352 277.234,299.828 282.745,305.304 288.255,310.78 293.766,316.256 299.276,321.732 
 304.787,327.208 310.298,327.208 315.808,327.208 321.319,327.208 326.829,327.208 332.34,327.208 337.85,327.208 343.361,327.208 348.872,327.208 354.382,327.208 
 359.893,327.208 365.403,327.208 370.914,327.208 376.424,327.208 381.935,327.208 387.445,327.208 392.956,327.208 398.467,327.208 403.977,327.208 409.488,327.208 
 414.998,327.208 420.509,327.208 426.019,327.208 431.53,327.208 437.04,327.208 442.551,327.208 448.062,327.208 453.572,327.208 459.083,327.208 464.593,327.208 
 470.104,327.208 475.614,327.208 481.125,327.208 486.635,327.208 492.146,327.208 497.657,327.208 503.167,327.208 508.678,327.208 514.188,327.208 519.699,327.208 
 525.209,327.208 530.72,327.208 536.23,327.208 541.741,327.208 547.252,327.208 552.762,327.208 558.273,327.208 563.783,327.208 569.294,327.208 574.804,327.208 
 580.315,327.208 
 "/>
 
 
 
 
 call 
 
 
 
 put

In [5]:
plot(Sm,StraddleProfit,color=:blue,linewidth=2,label="call+put")
plot!(ylim=(-1,5))
title!("Profit a Stradle, strike = $K")
xlabel!("Asset price at expiration")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 Profit a Stradle, strike = 5 
 
 
 Asset price at expiration 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:2; stroke-opacity:1; fill:none" points="
 29.2591,75.3053 34.7697,80.7815 40.2803,86.2576 45.7908,91.7338 51.3014,97.2099 56.8119,102.686 62.3225,108.162 67.833,113.638 73.3436,119.115 78.8542,124.591 
 84.3647,130.067 89.8753,135.543 95.3858,141.019 100.896,146.495 106.407,151.971 111.918,157.448 117.428,162.924 122.939,168.4 128.449,173.876 133.96,179.352 
 139.47,184.828 144.981,190.305 150.491,195.781 156.002,201.257 161.513,206.733 167.023,212.209 172.534,217.685 178.044,223.162 183.555,228.638 189.065,234.114 
 194.576,239.59 200.086,245.066 205.597,250.542 211.108,256.018 216.618,261.495 222.129,266.971 227.639,272.447 233.15,277.923 238.66,283.399 244.171,288.875 
 249.681,294.352 255.192,299.828 260.703,305.304 266.213,310.78 271.724,316.256 277.234,321.732 282.745,327.208 288.255,332.685 293.766,338.161 299.276,343.637 
 304.787,349.113 310.298,343.637 315.808,338.161 321.319,332.685 326.829,327.208 332.34,321.732 337.85,316.256 343.361,310.78 348.872,305.304 354.382,299.828 
 359.893,294.352 365.403,288.875 370.914,283.399 376.424,277.923 381.935,272.447 387.445,266.971 392.956,261.495 398.467,256.018 403.977,250.542 409.488,245.066 
 414.998,239.59 420.509,234.114 426.019,228.638 431.53,223.162 437.04,217.685 442.551,212.209 448.062,206.733 453.572,201.257 459.083,195.781 464.593,190.305 
 470.104,184.828 475.614,179.352 481.125,173.876 486.635,168.4 492.146,162.924 497.657,157.448 503.167,151.971 508.678,146.495 514.188,141.019 519.699,135.543 
 525.209,130.067 530.72,124.591 536.23,119.115 541.741,113.638 547.252,108.162 552.762,102.686 558.273,97.2099 563.783,91.7338 569.294,86.2576 574.804,80.7815 
 580.315,75.3053 
 "/>
 
 
 
 
 call+put

# Put-Call Parity for European Options

A no-arbitrage condition says that 
$$
C-P=e^{-my}(F-K)
$$
must hold. 

When the underying asset has no dividends (until expiration of the option), then $F=e^{my}S$.

In [6]:
S = 42                     #current price of underlying etc                       
K = 42
m = 0.5
y = 0.05
σ = 0.2

C = 2.893                  #assume this is the price of a call option(K)

F = exp(m*y)*S
P = C - exp(-m*y)*(F-K)

println("If C=$C, then P=$(round(P,3)) (at the same strike price)")

If C=2.893, then P=1.856 (at the same strike price)


# Pricing Bounds

The pricing bounds for (American and European) call options are

$\begin{align}
C  &  \leq e^{-my}F\\
C  &  \geq\max[0,e^{-my}(F-K)]
\end{align}$

The pricing bounds for (European) put options are

$\begin{align}
P_{E}  &  \leq e^{-my}K\\
e^{-my}(K-F)  &  \leq P_{E}
\end{align}$

In [7]:
S = 42
m = 0.5
y = 0.05
K = 30:0.5:50
n = length(K)

F = exp(m*y)*S

C_Upper = exp(-m*y)*F
C_Lower = max.(0,exp(-m*y)*(F.-K))

P_Upper = exp(-m*y)*K
P_Lower = max.(0,exp(-m*y)*(K.-F))

println()

In [8]:
plot(K,C_Upper*ones(n),color=:green,linewidth=2,line=(:dash),label=" C <= exp(-m*y)F ")
plot!(K,C_Lower,color=:blue,linewidth=2,line=(:dot,2),label=" C >= max[0,exp(-m*y)(F-K)]  ")
plot!(ylim=(-1,S+1))
title!("Price Bounds on European Call Options")
xlabel!("Strike price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 30 
 
 
 35 
 
 
 40 
 
 
 45 
 
 
 50 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 Price Bounds on European Call Options 
 
 
 Strike price 
 
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 28.9273,38.9635 42.712,38.9635 56.4967,38.9635 70.2814,38.9635 84.0661,38.9635 97.8507,38.9635 111.635,38.9635 125.42,38.9635 139.205,38.9635 152.99,38.9635 
 166.774,38.9635 180.559,38.9635 194.344,38.9635 208.128,38.9635 221.913,38.9635 235.698,38.9635 249.482,38.9635 263.267,38.9635 277.052,38.9635 290.836,38.9635 
 304.621,38.9635 318.406,38.9635 332.191,38.9635 345.975,38.9635 359.76,38.9635 373.545,38.9635 387.329,38.9635 401.114,38.9635 414.899,38.9635 428.683,38.9635 
 442.468,38.9635 456.253,38.9635 470.037,38.9635 483.822,38.9635 497.607,38.9635 511.392,38.9635 525.176,38.9635 538.961,38.9635 552.746,38.9635 566.53,38.9635 
 580.315,38.9635 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="1, 2" points="
 28.9273,257.457 42.712,261.098 56.4967,264.74 70.2814,268.382 84.0661,272.023 97.8507,275.665 111.635,279.306 125.42,282.948 139.205,286.589 152.99,290.231 
 166.774,293.872 180.559,297.514 194.344,301.156 208.128,304.797 221.913,308.439 235.698,312.08 249.482,315.722 263.267,319.363 277.052,323.005 290.836,326.646 
 304.621,330.288 318.406,333.93 332.191,337.571 345.975,341.213 359.76,344.854 373.545,348.496 387.329,352.137 401.114,352.598 414.899,352.598 428.683,352.598 
 442.468,352.598 456.253,352.598 470.037,352.598 483.822,352.598 497.607,352.598 511.392,352.598 525.176,352.598 538.961,352.598 552.746,352.598 566.53,352.598 
 580.315,352.598 
 "/>
 
 
 
 
 C <= exp(-m*y)F 
 
 
 
 C >= max[0,exp(-m*y)(F-K)]

In [9]:
plot(K,P_Upper,color=:green,linewidth=2,line=(:dash),label=" P <= exp(-m*y)K ")
plot!(K,P_Lower,color=:blue,linewidth=2,line=(:dot,2),label=" P >= max[0,exp(-m*y)(K-F)]  ")
plot!(ylim=(-1,50))
title!("Price Bounds on European Put Options")
xlabel!("Strike price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 30 
 
 
 35 
 
 
 40 
 
 
 45 
 
 
 50 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 Price Bounds on European Put Options 
 
 
 Strike price 
 
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 28.9273,165.119 42.712,161.977 56.4967,158.835 70.2814,155.694 84.0661,152.552 97.8507,149.41 111.635,146.268 125.42,143.127 139.205,139.985 152.99,136.843 
 166.774,133.701 180.559,130.56 194.344,127.418 208.128,124.276 221.913,121.135 235.698,117.993 249.482,114.851 263.267,111.709 277.052,108.568 290.836,105.426 
 304.621,102.284 318.406,99.1424 332.191,96.0006 345.975,92.8589 359.76,89.7172 373.545,86.5754 387.329,83.4337 401.114,80.292 414.899,77.1502 428.683,74.0085 
 442.468,70.8668 456.253,67.725 470.037,64.5833 483.822,61.4415 497.607,58.2998 511.392,55.1581 525.176,52.0163 538.961,48.8746 552.746,45.7329 566.53,42.5911 
 580.315,39.4494 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="1, 2" points="
 28.9273,353.623 42.712,353.623 56.4967,353.623 70.2814,353.623 84.0661,353.623 97.8507,353.623 111.635,353.623 125.42,353.623 139.205,353.623 152.99,353.623 
 166.774,353.623 180.559,353.623 194.344,353.623 208.128,353.623 221.913,353.623 235.698,353.623 249.482,353.623 263.267,353.623 277.052,353.623 290.836,353.623 
 304.621,353.623 318.406,353.623 332.191,353.623 345.975,353.623 359.76,353.623 373.545,353.623 387.329,353.623 401.114,350.878 414.899,347.737 428.683,344.595 
 442.468,341.453 456.253,338.312 470.037,335.17 483.822,332.028 497.607,328.886 511.392,325.745 525.176,322.603 538.961,319.461 552.746,316.319 566.53,313.178 
 580.315,310.036 
 "/>
 
 
 
 
 P <= exp(-m*y)K 
 
 
 
 P >= max[0,exp(-m*y)(K-F)]